In [ ]:
import sqlite3
import pandas as pd

import pystac_client
import odc.stac

import os

In [ ]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [ ]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [ ]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [146.34, -39.05, 146.44, -39.15]

# Set a start and end date
start_date = "1988-01-01"
end_date = "1988-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

In [ ]:
# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

items

In [ ]:
ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds


In [ ]:
# Print the loaded dataset
print("Dataset info:")
print(ds)

# Print the type of the bs_pc_10 data
print("\nType of bs_pc_10:")
print(type(ds.bs_pc_10))

# Print the data type (dtype) of the band values
print("\nData type of bs_pc_10 values:")
print(ds.bs_pc_10.dtype)

In [ ]:
ds.bs_pc_10[0].mean().values

In [ ]:
ds.bs_pc_10[1]

In [ ]:
ds.bs_pc_10[0][0]

In [ ]:
bbox = [146.29, -39.10, 146.39, -39.00]

# Set a start and end date
start_date = "2015-01-01"
end_date = "2020-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")


ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds

In [ ]:
ds.bs_pc_10.__len__()

In [ ]:
ds.bs_pc_10[5]

In [ ]:
ds.bs_pc_10[5][5]

In [ ]:
ds.pv_pc_50[5][5]

In [ ]:
ds.pv_pc_50[5].mean().values

In [ ]:
for item in ds:

    if item == 'qa':
        continue

    mean = ds[item].mean().values

    print(f"Mean value of {item}: {mean}")

In [ ]:
# Check if the operating system is Linux
if os.name == 'posix':
    db_path = '/data/oa3802fa25/GlobalMeltdown/fire.db'
else:
    db_path = 'fire.db'

# Connect to the appropriate database
conn = sqlite3.connect(db_path)

# Create a cursor object
cursor = conn.cursor()

# fetch data from fire.db
cursor.execute('''SELECT * from ground_data LIMIT 10''')
cursor.fetchall()


In [ ]:
# function to update each row in the data base table
def update_ground_data(cursor, conn, data):
    cursor.execute('''
    UPDATE ground_data
    SET bs_pc_10 = ?, bs_pc_50 = ?, bs_pc_90 = ?, pv_pc_10 = ?, pv_pc_50 = ?, pv_pc_90 = ?, npv_pc_10 = ?, npv_pc_50 = ?, npv_pc_90 = ?
    WHERE grid_id = ?
    ''', (
        data['bs_pc_10'], data['bs_pc_50'], data['bs_pc_90'],
        data['pv_pc_10'], data['pv_pc_50'], data['pv_pc_90'],
        data['npv_pc_10'], data['npv_pc_50'], data['npv_pc_90'],
        data['grid_id']
    ))

    # Commit the changes
    conn.commit()

In [ ]:
data_entry = {
    'time': '1987-01',
    'grid_id': 5831,
    'bs_pc_10': float(ds.bs_pc_10.mean().values),
    'bs_pc_50': float(ds.bs_pc_50.mean().values),
    'bs_pc_90': float(ds.bs_pc_90.mean().values),
    'pv_pc_10': float(ds.pv_pc_10.mean().values),
    'pv_pc_50': float(ds.pv_pc_50.mean().values),
    'pv_pc_90': float(ds.pv_pc_90.mean().values),
    'npv_pc_10': float(ds.npv_pc_10.mean().values),
    'npv_pc_50': float(ds.npv_pc_50.mean().values),
    'npv_pc_90': float(ds.npv_pc_90.mean().values)
}

update_ground_data(cursor, conn, data_entry)

In [ ]:
# fetch data from fire.db
cursor.execute('''SELECT * from ground_data LIMIT 10''')
cursor.fetchall()